<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [16]:
%%writefile going_modular/get_data.py

import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If th image folder doesn't exist, download it or else skip it
if image_path.is_dir():
    print(f"{image_path} already exists, skipping download")

else:
    print(f"Did not find {image_path}, Creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
# Download pizza, steak and sushi
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print(f"Downloading data....")
    f.write(request.content)

# Unzip pizza, steak data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r")as zip_ref:
    print(f"Unzipping pizza, steak, sushi...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Overwriting going_modular/get_data.py


In [17]:
# Example running of get_data.py
!python going_modular/get_data.py

data\pizza_steak_sushi already exists, skipping download
Unzipping pizza, steak, sushi...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [23]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import argparse
import torch
import  data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
parser = argparse.ArgumentParser(description="Getting some hyperparmeters.")

parser.add_argument("--num_epochs", type=int, default=10, help="number of epochs for the model to train")
parser.add_argument("--hidden_units", type=int, default=10, help="number of hidden units in hidden_layers")
parser.add_argument("--batch_size", type=int, default=32, help="batch size for our dataloaders")
parser.add_argument("--learning_rate", type=float, default=1e-3, help="learning rate for our optimizer")

parser.add_argument("--train_dir", type=str, default="data/pizza_steak_sushi/train", help="directory file path containing training data in standard image classification format")
parser.add_argument("--test_dir", type=str, default="data/pizza_steak_sushi/test", help="directory file path containing test data in standard image classification format")

# Get our arguments from the parser
args = parser.parse_args()

# Setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE} using {HIDDEN_UNITS} hidden units and a learning rate of {LEARNING_RATE}")

# Setup directories
train_dir = args.train_dir
test_dir = args.test_dir
print(f"[INFO] Training data file: {train_dir}")
print(f"[INFO] Testing data file: {test_dir}")

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

if __name__ == "__main__":
    # Create transforms
    data_transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])

    # Create DataLoaders with help from data_setup.py
    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=train_dir,
        test_dir=test_dir,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    # Create model with help from model_builder.py
    model = model_builder.TinyVGG(
        input_shape=3,
        hidden_units=HIDDEN_UNITS,
        output_shape=len(class_names)
    ).to(device)

    # Set loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=LEARNING_RATE)

    # Start training with help from engine.py
    engine.train(model=model,
                 train_dataloader=train_dataloader,
                 test_dataloader=test_dataloader,
                 loss_fn=loss_fn,
                 optimizer=optimizer,
                 epochs=NUM_EPOCHS,
                 device=device)

    # Save the model with help from utils.py
    utils.save_model(model=model,
                     target_dir="models",
                     model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_modular/train.py


In [24]:
# Example running of train.py
!python going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

[INFO] Training a model for 5 epochs with batch size 128 using 128 hidden units and a learning rate of 0.0003
[INFO] Training data file: data/pizza_steak_sushi/train
[INFO] Testing data file: data/pizza_steak_sushi/test
[INFO] Training a model for 5 epochs with batch size 128 using 128 hidden units and a learning rate of 0.0003
[INFO] Training data file: data/pizza_steak_sushi/train
[INFO] Testing data file: data/pizza_steak_sushi/test
[INFO] Training a model for 5 epochs with batch size 128 using 128 hidden units and a learning rate of 0.0003
[INFO] Training data file: data/pizza_steak_sushi/train
[INFO] Testing data file: data/pizza_steak_sushi/test
[INFO] Training a model for 5 epochs with batch size 128 using 128 hidden units and a learning rate of 0.0003
[INFO] Training data file: data/pizza_steak_sushi/train
[INFO] Testing data file: data/pizza_steak_sushi/test
[INFO] Training a model for 5 epochs with batch size 128 using 128 hidden units and a learning rate of 0.0003
[INFO] Tra


100%|##########| 5/5 [04:46<00:00, 57.36s/it]


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [28]:
%%writefile going_modular/predict.py
import torch
import torchvision

import model_builder
import argparse

# Create image and model_pth arguments for our predict script
parser = argparse.ArgumentParser()
parser.add_argument("--image",
                    help="target_image filepath to predict on")

parser.add_argument("--model_pth",
                    default="models/05_going_modular_script_mode_tinyvgg_model.pth",
                    help="target_model to usr for prediction")


args = parser.parse_args()

# Setup class names
class_names = ["pizza", "steak", "sushi"]

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# SEt image path
image_path = args.image
print(f"[INFO] Predicting on {image_path}")

# Function to load in the model
def load_model(filepath=args.model_pth):
    # Need to use same hyperparameters as saved model
    model = model_builder.TinyVGG(input_shape=3,
                                  hidden_units=128,
                                  output_shape=3).to(device)

    print(f"[INFO] Loading in model from: {filepath}")
    # Load in the saved model state dictionary from file
    model.load_state_dict(torch.load(filepath))
    return model

# Function to load in model + predict on select image
def predict_on_image(image_path=image_path, filepath=args.model_pth):
    # Load the model
    model = load_model(filepath)

    # Load in the image and turn it into torch.float32 (same type as model)
    image = torchvision.io.read_image(str(image_path)).type(torch.float32)

    # Preprocess the image to get it between 0 and 1
    image = image / 255.

    # Resize the image to be the same size as the model
    transform = torchvision.transforms.Resize(size=(64, 64))
    image = transform(image)

    # Predict on image
    model.eval()
    with torch.inference_mode():
        # Put image to target device
        image = image.to(device)

        # Get pred logits
        pred_logits = model(image.unsqueeze(dim=0)) # make sure image has batch dimension (shape: [batch_size, height, width, color_channels])

        # Get pred probs
        pred_prob = torch.softmax(pred_logits, dim=1)

        # Get pred labels
        pred_label = torch.argmax(pred_prob, dim=1)
        pred_label_class = class_names[pred_label]

    print(f"[INFO] Pred class: {pred_label_class}, Pred prob: {pred_prob.max():.3f}")

if __name__ == "__main__":
    predict_on_image()

Overwriting going_modular/predict.py


In [29]:
# Example running of predict.py 
!python going_modular/predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Predicting on data/pizza_steak_sushi/test/sushi/175783.jpg
[INFO] Loading in model from: models/05_going_modular_script_mode_tinyvgg_model.pth
[INFO] Pred class: pizza, Pred prob: 0.478
